# 8.2 Scenario analyses with intervals

In this tutorial we will work through an example where we create 'credible intervals'. We will return to the example from tutorial 5.2 where we assessed a vaccine for SIR, but this time we will incorporate some of the stochasticity that we saw in tutorial 5.5. We will also illustrate how you can add demographics to your model.

In [ ]:
import numpy as np # Math
import sciris as sc # Utilities
import pylab as pl # Plotting
import starsim as ss # ABM


class Vaccine(ss.Intervention):
    def __init__(self, timestep=10, prob=0.5, imm_boost=2.0):
        super().__init__() # Initialize the intervention
        self.timestep = timestep # Store the timestep the vaccine is applied on
        self.prob = prob # Store the probability of vaccination
        self.imm_boost = imm_boost # Store the amount by which immunity is boosted

    def apply(self, sim): # Apply the vaccine
        if sim.ti == self.timestep: # Only apply on the matching timestep
            sis = sim.diseases.sis # Shorten the name of the disease module
            eligible_ids = sim.people.uid[sis.susceptible] # Only susceptible people are eligible
            n_eligible = len(eligible_ids)  # Number of people who are eligible
            to_vaccinate = self.prob > np.random.rand(n_eligible) # Define which of the n_eligible people get vaccinated
            vaccine_ids = eligible_ids[to_vaccinate]
            sis.immunity[vaccine_ids] += self.imm_boost


def make_sim(seed=1, n_timesteps=100, use_vaccine=False, timestep=10, prob=0.5, imm_boost=2.0):
    """ Make the simulation, but do not run it yet """

    pars = dict(
        n_agents = 1000,
        start = 0,
        end = n_timesteps,
        dt = 1.0,
        verbose = 0,
        rand_seed = seed,
        networks = 'random',
        diseases = dict(
            type = 'sis',
            beta = 0.05,
            waning = 0.05,
        )
    )

    # Define "baseline" and "intervention" sims without and with the vaccine
    if use_vaccine:
        vaccine = Vaccine(timestep=timestep, prob=prob, imm_boost=imm_boost)
        sim = ss.Sim(pars, interventions=vaccine)
    else:
        sim = ss.Sim(pars)

    return sim


# Prepare to run multiple times
n_seeds = 20  # Don't use too many here or your sim will take a very long time!
n_timesteps = 100  # Again, don't use too many
baseline_results = np.empty((n_seeds, n_timesteps+1))  # Initialize storage of baseline results
vaccine_results  = np.empty((n_seeds, n_timesteps+1))  # Initialize storage of vaccine results
difference_results = np.empty(n_seeds)  # Initialize storage of differences - this will tell us the impact
baseline_sims = [] # Initialize the list of baseline simulations
vaccine_sims = [] # Initialize the list of baseline simulations

# Make the simulations with different seeds
for seed in range(n_seeds): # Run over 5 different random seeds
    baseline_sim = make_sim(seed=seed, n_timesteps=n_timesteps) # Run the simulation with no vaccine
    vaccine_sim  = make_sim(seed=seed, n_timesteps=n_timesteps, use_vaccine=True) # Run the simulation with the vaccine
    baseline_sims.append(baseline_sim) # Add the baseline sim to the list
    vaccine_sims.append(vaccine_sim) # Add the vaccine sim to the list

def run_sim(sim):
    """ Run the simulation and return the results """
    sim.run()
    results = sc.objdict()
    results.time = sim.yearvec
    results.n_infected = sim.results.sis.n_infected
    return results

# Run the simulations in parallel
baseline_sim_results = sc.parallelize(run_sim, baseline_sims) # Run baseline sims
vaccine_sim_results  = sc.parallelize(run_sim, vaccine_sims) # Run the vaccine sims


# Pull out the results
for seed in range(n_seeds):
    baseline = baseline_sim_results[seed]
    vaccine = vaccine_sim_results[seed]
    baseline_results[seed, :] = baseline.n_infected # Pull out results from baseline
    vaccine_results[seed, :] = vaccine.n_infected  # Pull out results from vaccine scenarios
    difference_results[seed] = baseline_results[seed, :].sum() - vaccine_results[seed, :].sum()  # Calculate differences

# Get the qunatiles for plotting
lower_bound_baseline = np.quantile(baseline_results, 0.05, axis=0)
median_baseline      = np.quantile(baseline_results, 0.5, axis=0)
upper_bound_baseline = np.quantile(baseline_results, 0.95, axis=0)
lower_bound_vaccine  = np.quantile(vaccine_results, 0.05, axis=0)
median_vaccine       = np.quantile(vaccine_results, 0.5, axis=0)
upper_bound_vaccine  = np.quantile(vaccine_results, 0.95, axis=0)

# Get the time vector for plotting
time = baseline.time

# Calculate differences
lower_bound_diff = np.quantile(difference_results, 0.05)
upper_bound_diff = np.quantile(difference_results, 0.95)
median_diff = np.quantile(difference_results, 0.5)
title = f'Estimated impact: {median_diff:.0f} (90% CI: {lower_bound_diff:.0f}, {upper_bound_diff:.0f}) infections averted'


# Do the plotting
pl.figure()  # Create the figure
pl.title(title)
pl.fill_between(time, lower_bound_baseline, upper_bound_baseline, alpha=0.5) # Plot the uncertainty bound for baseline
pl.plot(time, median_baseline, label='Baseline') # Plot the median for baseline
pl.fill_between(time, lower_bound_vaccine, upper_bound_vaccine, alpha=0.5) # Plot the uncertainty bound for vaccine
pl.plot(time, median_vaccine, label='With vaccine') # Plot the median for vaccine
pl.xlabel('Time')
pl.ylabel('Number of people infected')
pl.legend() # Add legend
pl.ylim(bottom=0) # Star the y-axis at 0
pl.xlim(left=0) # Start the x-axis at 0
pl.show()




# Problem 2:

In this section we'll see how to use the Starsim vaccine intervention and add basic demographics.

In [ ]:
import sciris as sc # Utilities
import pylab as pl # Plotting
import starsim as ss # ABM

pars = sc.objdict(
    n_agents = 1000,
    birth_rate = 20,
    death_rate = 15,
    networks = sc.objdict(
        type = 'randomnet',
        n_contacts = 4,
    ),
    diseases = sc.objdict(
        type = 'sir',
        dur_inf = 10,
        beta = 0.1,
    )
)
sim_base = ss.Sim(pars=pars)
sim_base.run()

# Create an ongoing vaccine intervention 
my_vax = ss.sir_vaccine(pars=dict(efficacy=0.5))
intv = ss.routine_vx(start_year=2015, prob=0.2, product=my_vax)
sim_intv = ss.Sim(pars=pars, interventions=intv)
sim_intv.run()

# Make plots
pl.figure()
pl.plot(sim_base.yearvec, sim_base.results.sir.prevalence, label='Baseline')
pl.plot(sim_intv.yearvec, sim_intv.results.sir.prevalence, label='Vax')
pl.axvline(x=2015, color='k', ls='--')
pl.title('Prevalence')
pl.legend()
pl.show()


# Problem 3

Here we'll see an example showing how to add more detailed country-specific demographics. Your task here is to think about how you would get these data for your country.

In [ ]:
import pandas as pd  # Data analysis
import sciris as sc # Utilities
import starsim as ss  # ABM
import pylab as pl # Plotting


def make_run_sim(location='zimbabwe'):
    # Make demographic modules
    fertility_rates = {'fertility_rate': pd.read_csv(f'data/{location}_asfr.csv')}
    pregnancy = ss.Pregnancy(pars=fertility_rates)
    death_rates = {'death_rate': pd.read_csv(f'data/{location}_deaths.csv'), 'units': 1}
    death = ss.Deaths(death_rates)

    # Make people with age structure
    people = ss.People(1000, age_data=pd.read_csv(f'data/{location}_age.csv'))
    total_pop_1990 = 9980999

    # Make parameters
    pars = dict(
        dt=1,
        total_pop=total_pop_1990,
        start=1990,
        n_years=40,
        people=people,
        demographics=[pregnancy, death]
    )

    sim = ss.Sim(**pars)
    sim.run()

    # Create the figure
    pl.figure()

    # Create the first panel in the plot
    pl.subplot(2,1,1)  
    pl.title('Population size')
    pl.plot(sim.yearvec, sim.results.n_alive)
    sc.commaticks() # Use full numbers rather than SI notation for the y-axis

    # Create the second panel in the plot
    pl.subplot(2,1,2)  
    pl.title('Births')
    pl.plot(sim.yearvec, sim.results.pregnancy.births / sim.dt)
    sc.commaticks() # Use full numbers rather than SI notation for the y-axis

    sc.figlayout() # Automatically readjust panel positions


make_run_sim()